In [2]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import scipy as sp

# Task easy

$ y'' + e (y^2 - 1) y' + y = 0 $

$ x' = z, z' = e (1 - x^2) z - x $ 

$ x(0) = 2, z(0) = 0, 0<t<100, 1/100 < e < 100 $

In [20]:
eps = 1

def f_calc(x, z):
    return np.array((z, eps*(1-x**2)*z-x))

def f_calc_v(y, t):
    x,z = y
    return np.array((z, eps*(1-x**2)*z-x))

In [27]:
y = sp.integrate.odeint(f_calc_v, np.array((2,0)), np.linspace(0, 100, 201, endpoint=True))
px.line(y=y[:,0]).add_trace(go.Scatter(y=y[:,1]))

### explicit Runge-Kutt - Модифицированный метод Эйлера с пересчетом

![Alt text](explic_rk1.png)

In [32]:
def calc_expl_rk(n):
    tmax = 100
    tmin = 0
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    y = np.zeros((n+1, 2))
    # return t.shape, y[0].shape
    y[0] = (2, 0)
    for i in range(1, n+1):
        y1 = y[i-1] + h * f_calc(*y[i-1])
        y[i] = y[i-1] + (h/2 * (f_calc(*y[i-1])+f_calc(*y1)))
    return t, y 

t, y = calc_expl_rk(200)
# df = pd.DataFrame({'t': t, 'x': y[:,0], 'z': y[:,1]})
px.line(x=t, y=y[:,0]).add_trace(go.Scatter(x=t, y=y[:,1]))

## explicit Adams' method

$ % http://w.ict.nsc.ru/books/textbooks/akhmerov/nm-ode_unicode/1-6.html $ 

![Alt text](adams_2nd.png)

y' = f(y)
y1 = y0 + y'*h + y'' * h^2/2 + O(h^3)


In [34]:
def runge_kutt_iter(y0, h): 
    y1 = y0 + h * f_calc(*y0)
    y1 = y0 + (h/2 * (f_calc(*y0)+f_calc(*y1)))
    return y1

def calc_adams(n):
    tmax = 100
    tmin = 0
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    y = np.zeros((n+1, 2))
    y[0] = (2, 0)
    y[1] = runge_kutt_iter(y[0], h)
    for i in range(2, n+1):
        y[i] = y[i-1] + h/2 * (3 * f_calc(*y[i-1]) - f_calc(*y[i-2]))
    return t, y

t, y = calc_expl_rk(200)
# df = pd.DataFrame({'t': t, 'x': y[:,0], 'z': y[:,1]})
px.line(x=t, y=y[:,0]).add_trace(go.Scatter(x=t, y=y[:,1]))

## Backwards differentiating formula

y[n+1] = (4/3) * y[n] - (1/3) * y[n-1] + (2/3) * h * f(t[n+1], y[n+1])

or

y[n] = (4/3) * y[n-1] - (1/3) * y[n-2] + (2/3) * h * f(t[n], y[n])

In [36]:
def calc_bdf(n):
    tmax = 100
    tmin = 0
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    y = np.zeros((n+1, 2))
    y[0] = (2, 0)
    y[1] = runge_kutt_iter(y[0], h)
    for i in range(2, n+1):
        to_solve = lambda yi: -yi + (4/3) * y[i-1] - (1/3) * y[i-2] + (2/3) * h * f_calc(*yi)
        y[i] = sp.optimize.fsolve(to_solve, y[i-1])
    return t, y

t, y = calc_expl_rk(200)
# df = pd.DataFrame({'t': t, 'x': y[:,0], 'z': y[:,1]})
px.line(x=t, y=y[:,0]).add_trace(go.Scatter(x=t, y=y[:,1]))

# Task hard

![Alt text](hardtask.png)

In [3]:
eps = 0.01
y0 = np.array((3, 15, 0.01))

def f_calc(x, y, a):
    return np.array(
        (x*(1 - 0.5*x - 2/7/a**2 * y),
         y*(2*a - 3.5*a**2*x - 0.5*y),
         eps*(2 - 7*a*x))
    )

def f_calc_v(y, t):
    return f_calc(*y)

In [5]:
y = sp.integrate.odeint(f_calc_v, y0, np.linspace(0, 1500, 20000, endpoint=True), full_output=False)
# y = np.clip(y, None, 1e6)
px.line(y=y[:,0]).add_trace(go.Scatter(y=y[:,1])).add_trace(go.Scatter(y=y[:,2]))

## Implicit runge-kutt

### SDIRK method

![Alt text](sdirk.png)

Source: https://www.mathnet.ru/links/7408ea1523279fa5eef2e6e06d201072/mm653.pdf

y[n+1] = y[n] + h * sum(b[i] * k[i]), where:

$ k_i = f(t_n + c_i*h, y_n + h*\sum_{j=1}^2{a_{i,j}*k_j} ), i=1,2 $


In [6]:
def calc_implicit_rk(n):
    g = 1 - 2**0.5 / 2
    c = np.array((g, 1))
    a = np.array((
        (g, 0),
        (1-g, g)
    ))
    b = np.array((1-g, g))

    tmax = 1500
    tmin = 0
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    y = np.zeros((n+1, 3))
    y[0] = y0
    k = np.zeros((2,3))

    def to_call(yn):
        def to_solve(flat_k):
            k = flat_k.reshape((2,3))
            ans =  np.array((
                k[0] - f_calc(yn[0] + h * sum(a[0]*k[:,0]), yn[1] + h * sum(a[0]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])),
                k[1] - f_calc(yn[0] + h * sum(a[1]*k[:,0]), yn[1] + h * sum(a[1]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2]))
            ))
            # print(ans)
            return ans.flatten()
        return to_solve

    for i in range(1, n+1):
        to_solve = to_call(y[i-1])
        k = sp.optimize.fsolve(to_solve, [1]*6).reshape((2,3))
        y[i][0] = y[i-1][0] + h*sum(b*k[:,0])
        y[i][1] = y[i-1][1] + h*sum(b*k[:,1])
        y[i][2] = y[i-1][2] + h*sum(b*k[:,2])
    return t, y 


t, y = calc_implicit_rk(2000)
# y = np.clip(y, None, 1e6)
# df = pd.DataFrame({'t': t, 'x': y[:,0], 'z': y[:,1]})
px.line(x=t, y=y[:,0]).add_trace(go.Scatter(x=t, y=y[:,1])).add_trace(go.Scatter(x=t, y=y[:,2]))


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.

